In [1]:
import os 
import pandas as pd
path = '/Users/bokie/code/python/CMS/可转债套利/cms/daily_track'
dates = os.listdir(path)
dates.remove('.DS_Store')
dates.sort()
print('dates \n',dates)
daily_csvs = os.listdir(path+f'/{dates[-1]}')
pair_list = [i for i in daily_csvs if len(i.split('@'))==2]
print(dates[-1],'pair_list \n',pair_list)

dates 
 ['20220909', '20220913', '20220914', '20220915', '20220916', '20220919', '20220920', '20220921', '20220922', '20220923', '20220926', '20220927', '20220928', '20220929', '20220930', '20221010', '20221011', '20221012', '20221013', '20221014', '20221017', '20221018', '20221019', '20221020', '20221021', '20221024', '20221025', '20221026', '20221027', '20221028', '20221031', '20221101', '20221102', '20221103', '20221104', '20221107', '20221108', '20221109', '20221110', '20221111', '20221114', '20221115', '20221116', '20221117', '20221118', '20221121', '20221122', '20221123', '20221124']
20221124 pair_list 
 ['123067.SZ@300382.SZ.csv', '128081.SZ@002203.SZ.csv', '113626.SH@603596.SH.csv', '128017.SZ@002597.SZ.csv', '110061.SH@600674.SH.csv', '128111.SZ@002738.SZ.csv', '113534.SH@603876.SH.csv', '123057.SZ@300586.SZ.csv', '123031.SZ@300655.SZ.csv', '113016.SH@601127.SH.csv', '123092.SZ@300332.SZ.csv', '123046.SZ@300587.SZ.csv', '128095.SZ@002812.SZ.csv', '128106.SZ@002840.SZ.csv', '11

In [2]:
pair_dic = {} #key为对子名，value为字典，字典的key为日期，value为该对子对应日期的dataframe
for date in dates:
    tmp_daily_csvs = os.listdir(path+f'/{date}')
    for i in tmp_daily_csvs:
            
        if len(i.split('@'))==2:

            if i not in pair_dic:
                pair_dic[i] = {}

            tmp_df = pd.read_csv(path+f'/{date}/{i}')
            # if len(tmp_df)==3:
            pair_dic[i][date] = (tmp_df)
            

In [3]:
print(pair_dic.keys())
print(len(pair_dic.keys()))
print(len(pair_dic['113642.SH@603185.SH.csv']))
pair_dic['113642.SH@603185.SH.csv']['20220909']

dict_keys(['128081.SZ@002203.SZ.csv', '113642.SH@603185.SH.csv', '128017.SZ@002597.SZ.csv', '110061.SH@600674.SH.csv', '113534.SH@603876.SH.csv', '123031.SZ@300655.SZ.csv', '113016.SH@601127.SH.csv', '123046.SZ@300587.SZ.csv', '123027.SZ@300487.SZ.csv', '113025.SH@601677.SH.csv', '113537.SH@603348.SH.csv', '128029.SZ@002078.SZ.csv', '113626.SH@603596.SH.csv', '128111.SZ@002738.SZ.csv', '123092.SZ@300332.SZ.csv', '128095.SZ@002812.SZ.csv', '123067.SZ@300382.SZ.csv', '123057.SZ@300586.SZ.csv', '128106.SZ@002840.SZ.csv'])
19
40


,Unnamed: 0,pair,cash,signal,bond price,stock price,bond hold,stock hold,commission,tax,reverse open,diff,asset value,net value,open/close
0,0,113642.SH@603185.SH,0,NaN,NaN,NaN,0,0,0,0,NaN,NaN,0,0,"0.023,0"


In [4]:
df_dic = {}
sharpe_ratio_dic = {}
df_sharpe_ratio = pd.DataFrame(data=[],columns=['sharpe ratio','track days'])
for k in pair_dic.keys():
    df_tmp = pd.DataFrame([])    
    df_tmp['date'] =  list(pair_dic[k].keys())
    df_tmp['date'] = pd.to_datetime(df_tmp['date'])

    df_tmp['net value'] = [v['net value'].tolist()[-1] for v in pair_dic[k].values()]
    df_tmp['net value cum'] = df_tmp['net value'].cumsum()

    df_tmp['bond price'] = [v['bond price'].tolist()[-1] for v in pair_dic[k].values()]
    df_tmp['return rate'] = df_tmp['net value']/(df_tmp['bond price']*2)
    # df_tmp['return rate'] = df_tmp['return rate'].fillna(0) + 1
    # df_tmp['return rate cum'] = df_tmp['return rate'].cumprod()
    
    #用净值的概念去算，一开始规定bond price两倍作为初始资金
    # r_i = x_i/x_(i-1) sharpe = [x_n/x_0 - 1]/std(r_i)
    
    init_capital = df_tmp['bond price'].dropna().tolist()[0]*2 
    df_tmp['capital'] = init_capital+df_tmp['net value cum']
    df_tmp['return'] = df_tmp['capital']/df_tmp['capital'].shift(1) - 1
    df_tmp['return rate cum'] = (df_tmp['return']+1).cumprod()
    total_return = df_tmp['capital'].iloc[-1]/df_tmp['capital'].iloc[0] - 1
    standard_deviation = df_tmp['capital'].std()
    sharpe_ratio = total_return/standard_deviation
    track_days = len(pair_dic[k])
    df_sharpe_ratio.loc[k] = [sharpe_ratio,track_days]
    df_dic[k]=df_tmp
    sharpe_ratio_dic[k] = sharpe_ratio

In [5]:
# df_sharpe_ratio

In [6]:
#按照夏普率大小排序
f = sorted(sharpe_ratio_dic.items(),key = lambda x:x[1],reverse = True)
print(f)

df_dic['128111.SZ@002738.SZ.csv']

[('110061.SH@600674.SH.csv', 0.009350546872870909), ('128029.SZ@002078.SZ.csv', 0.008659956171367384), ('123027.SZ@300487.SZ.csv', 0.005948200126647179), ('123067.SZ@300382.SZ.csv', 0.005014555409174037), ('123046.SZ@300587.SZ.csv', 0.004538174665302332), ('123092.SZ@300332.SZ.csv', 0.004350048513301556), ('128017.SZ@002597.SZ.csv', 0.0037204299770916306), ('113537.SH@603348.SH.csv', 0.0027534403462006183), ('113534.SH@603876.SH.csv', 0.0019505448712751037), ('128111.SZ@002738.SZ.csv', 0.0015701161427408444), ('123057.SZ@300586.SZ.csv', 0.0010099659716134525), ('123031.SZ@300655.SZ.csv', 0.000895896280281062), ('128095.SZ@002812.SZ.csv', 0.0008687575640356265), ('113016.SH@601127.SH.csv', 0.0008574425030788486), ('113642.SH@603185.SH.csv', 0.0003993347757940546), ('128081.SZ@002203.SZ.csv', -0.0011452355821979607), ('113626.SH@603596.SH.csv', -0.001736043001786478), ('113025.SH@601677.SH.csv', -0.001985289127768508), ('128106.SZ@002840.SZ.csv', -0.003898475074219544)]


,date,net value,net value cum,bond price,return rate,capital,return,return rate cum
0,2022-10-14,-29.221531,-29.221531,928.019,-0.015744,1826.816469,NaN,NaN
1,2022-10-17,4.004201,-25.217330,918.068,0.002181,1830.820670,0.002192,1.002192
2,2022-10-18,0.000000,-25.217330,NaN,NaN,1830.820670,0.000000,1.002192
3,2022-10-19,8.406080,-16.811250,931.868,0.004510,1839.226750,0.004591,1.006793
4,2022-10-20,11.418023,-5.393227,916.721,0.006228,1850.644773,0.006208,1.013044
5,2022-10-21,-5.441317,-10.834544,901.503,-0.003018,1845.203456,-0.002940,1.010065
6,2022-10-24,4.023908,-6.810636,902.100,0.002230,1849.227364,0.002181,1.012268
7,2022-10-25,12.237707,5.427072,891.070,0.006867,1861.465072,0.006618,1.018967
8,2022-10-26,-6.195254,-0.768182,903.900,-0.003427,1855.269818,-0.003328,1.015575
9,2022-10-27,11.013747,10.245565,909.399,0.006056,1866.283565,0.005936,1.021604


In [7]:
import plotly.graph_objs as go

traces = [go.Scatter(x=v['date'],y=v['return rate cum'],name=k,) for k,v in df_dic.items()]

trace = go.Scatter(
    x=df_tmp['date'],
    y=df_tmp['return rate cum'],
    name='return rate cum',
    # xaxis='x', 
    # yaxis='y2'#标明设置一个不同于trace1的一个坐标轴
)

data = [trace]
data = traces
# https://blog.csdn.net/weixin_58587245/article/details/127315679 template 可选参数
layout = go.Layout({
                #    "template": 'simple_white',
                #    "template": 'presentation',
                #    "template": 'seaborn',
                #    "template": 'ggplot2',
                   "title":{"text": 'return rate cum', "x": 0.5}, 
                   "xaxis":{"title": {"text": None}, "tickformat": ''},
                   "yaxis":{"title": {"text": ""}},
                   "yaxis2": {'anchor': 'x', "overlaying": 'y', "side": 'right'}, #设置坐标轴的格式，一般次坐标轴在右侧
                   "legend":{"title": {"text": ""}, "x": 0.9, "y": 1.1},
                   "width": 1200,
                   "height": 500})
 
fig = go.Figure(data=data, layout=layout)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=1.3
))

fig.show()